In [154]:
import glob 
from pandas import read_csv, concat

#diretório
input_dir = r"C:\Users\Natan\Documents\TPAD2\dados"

##### **Listando os arquivos** #####

In [155]:
arquivos_ies = glob.glob(r"C:\Users\Natan\Documents\TPAD2\dados/*IES*.CSV")
arquivos_cursos = glob.glob(r"C:\Users\Natan\Documents\TPAD2\dados/*Cadastro_Cursos*.CSV")

##### **Filtrando colunas** #####

In [156]:
colunas_ies = ['CO_IES','SG_IES','TP_CATEGORIA_ADMINISTRATIVA','TP_ORGANIZACAO_ACADEMICA']

#Diurno
colunas_cadastro = ['NU_ANO_CENSO','SG_UF','CO_IES','QT_ING_DIURNO','QT_CONC_DIURNO','TP_ORGANIZACAO_ACADEMICA','TP_CATEGORIA_ADMINISTRATIVA','NO_CINE_ROTULO']
#Noturno
colunas_cadastro_noturno = ['NU_ANO_CENSO','SG_UF','CO_IES','QT_ING_NOTURNO','QT_CONC_NOTURNO','TP_ORGANIZACAO_ACADEMICA','TP_CATEGORIA_ADMINISTRATIVA','NO_CINE_ROTULO']

##### **Leitura dos dados e Correção de tipagem** #####

In [157]:
#Arquivos da cadastro cursos - Diurno
tabela_cadastro = []

for arquivo in arquivos_cursos:
    tabela_cadastro.append(read_csv(arquivo, sep=';', encoding='ISO 8859-1',usecols=colunas_cadastro)
    .rename(columns = {'NU_ANO_CENSO':'ano'})
    .query('TP_ORGANIZACAO_ACADEMICA==1 & TP_CATEGORIA_ADMINISTRATIVA==1')
    .groupby(['ano', 'CO_IES','NO_CINE_ROTULO']).agg(
        ingressantes_diurno = ('QT_ING_DIURNO', 'sum'),
        concluintes_diurno =('QT_CONC_DIURNO', 'sum')
        #nome_curso=('NO_CINE_ROTULO','max'))
    ))

cdd = concat(tabela_cadastro).reset_index()

#Convertendo p/ inteiros
cdd['ingressantes_diurno'] = cdd['ingressantes_diurno'].astype(int)
cdd['concluintes_diurno'] = cdd['concluintes_diurno'].astype(int)

#Arquivos da cadastro ies 
tabela_ies = []

for arquivo in arquivos_ies:
    tabela_ies.append(read_csv(arquivo, sep=';', encoding='ISO 8859-1',usecols=colunas_ies)
    .query('TP_ORGANIZACAO_ACADEMICA==1 & TP_CATEGORIA_ADMINISTRATIVA==1')
    )

ies = concat(tabela_ies)

####################################################################

#Arquivos da cadastro cursos - Noturno
tabela_cadastro_noturno = []

for arquivo2 in arquivos_cursos:
    tabela_cadastro_noturno.append(read_csv(arquivo2, sep=';', encoding='ISO 8859-1',usecols=colunas_cadastro_noturno)
    .rename(columns = {'NU_ANO_CENSO':'ano'})
    .query('TP_ORGANIZACAO_ACADEMICA==1 & TP_CATEGORIA_ADMINISTRATIVA==1')
    .groupby(['ano', 'CO_IES','NO_CINE_ROTULO']).agg(ingressantes_noturno = ('QT_ING_NOTURNO', 'sum'), concluintes_noturno =('QT_CONC_NOTURNO', 'sum'))
    )

cdn = concat(tabela_cadastro_noturno).reset_index()

#Convertendo p/ inteiros
cdn['ingressantes_noturno'] = cdn['ingressantes_noturno'].astype(int)
cdn['concluintes_noturno'] = cdn['concluintes_noturno'].astype(int)

##### **Agrupando os dados por curso e ano, nessa ordem (apenas UFPB)** #####

In [197]:
####################################################################

#Evasão - Diurno *dfd = dataframe diurno
dfd = ies.merge(cdd, on='CO_IES', how='inner')
dfd = dfd.loc[dfd['SG_IES']=='UFPB']

dfd = (dfd
.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes_diurno=('ingressantes_diurno','sum'),
    concluintes_diurno=('concluintes_diurno','sum')
).reset_index())

####################################################################

#Evasão - Noturno *dfn = dataframe noturno
dfn = ies.merge(cdn, on='CO_IES', how='inner')
dfn = dfn.loc[dfn['SG_IES']=='UFPB']

dfn = (dfn
.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes_noturno=('ingressantes_noturno','sum'),
    concluintes_noturno=('concluintes_noturno','sum')
).reset_index())

##### **Filtrando os cursos do CCSA que possuem dados disponíveis** #####

In [207]:
#Diurnos
adm_dia   = dfd.loc[dfd['NO_CINE_ROTULO']=='Administração']
econo_dia = dfd.loc[dfd['NO_CINE_ROTULO']=='Economia']
arqi_dia  = dfd.loc[dfd['NO_CINE_ROTULO']=='Arquivologia']
bibl_dia  = dfd.loc[dfd['NO_CINE_ROTULO']=='Biblioteconomia']
atua_dia  = dfd.loc[dfd['NO_CINE_ROTULO']=='Ciências atuariais']
cont_dia  = dfd.loc[dfd['NO_CINE_ROTULO']=='Contabilidade']
gest_dia  = dfd.loc[dfd['NO_CINE_ROTULO']=='Gestão pública']
rela_dia  = dfd.loc[dfd['NO_CINE_ROTULO']=='Relações internacionais']

#Noturnos
adm_noi   = dfn.loc[dfn['NO_CINE_ROTULO']=='Administração']
econo_noi = dfn.loc[dfn['NO_CINE_ROTULO']=='Economia']
arqi_noi  = dfn.loc[dfn['NO_CINE_ROTULO']=='Arquivologia']
bibl_noi  = dfn.loc[dfn['NO_CINE_ROTULO']=='Biblioteconomia']
atua_noi  = dfn.loc[dfn['NO_CINE_ROTULO']=='Ciências atuariais']
cont_noi  = dfn.loc[dfn['NO_CINE_ROTULO']=='Contabilidade']
gest_noi  = dfn.loc[dfn['NO_CINE_ROTULO']=='Gestão pública']
rela_noi  = dfn.loc[dfn['NO_CINE_ROTULO']=='Relações internacionais']

##### **Comparando dois horários de um curso** #####

In [208]:
adm_dia = (adm_dia.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes_diurno=('ingressantes_diurno','sum'),
    concluintes_diurno=('concluintes_diurno','sum')
).assign(
    evasão = lambda x: (1-(x.concluintes_diurno/x.ingressantes_diurno.shift(5)))*100
))

adm_noi = (adm_noi.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes_noturno=('ingressantes_noturno','sum'),
    concluintes_noturno=('concluintes_noturno','sum')
).assign(
    evasão = lambda x: (1-(x.concluintes_noturno/x.ingressantes_noturno.shift(5)))*100
))

In [210]:
def consulta_curso():
    curso = str(input('Digite a sigla do curso de interesse: '))
    curso_dia = curso+'_dia'
    curso_noite = curso+'_noi' 
    print(curso_dia, curso_noite)

In [211]:
consulta_curso()

adm_dia adm_noi


In [215]:
econo_dia = (econo_dia.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes_diurno=('ingressantes_diurno','sum'),
    concluintes_diurno=('concluintes_diurno','sum')
).assign(
    evasão = lambda x: (1-(x.concluintes_diurno/x.ingressantes_diurno.shift(5)))*100
))

econo_noi = (econo_noi.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes_noturno=('ingressantes_noturno','sum'),
    concluintes_noturno=('concluintes_noturno','sum')
).assign(
    evasão = lambda x: (1-(x.concluintes_noturno/x.ingressantes_noturno.shift(5)))*100
))

In [216]:
econo_dia

ingressantes_diurno  concluintes_diurno     evasão
NO_CINE_ROTULO ano                                                     
Economia       2009                    0                   0        NaN
               2010                 1164                 360        NaN
               2011                 1152                 384        NaN
               2012                 1140                 288        NaN
               2013                  852                 360        NaN
               2014                 1020                 204       -inf
               2015                  984                 276  76.288660
               2016                  948                 336  70.833333
               2017                 1032                 432  62.105263
               2018                  852                 240  71.830986
               2019                  900                 312  69.411765
               2020                 1044                 132  86.585366

In [217]:
econo_noi

ingressantes_noturno  concluintes_noturno     evasão
NO_CINE_ROTULO ano                                                       
Economia       2009                     0                    0        NaN
               2010                  1248                  300        NaN
               2011                  1176                  216        NaN
               2012                  1152                  228        NaN
               2013                  1056                  204        NaN
               2014                  1020                  168       -inf
               2015                   984                   96  92.307692
               2016                   900                  276  76.530612
               2017                  1116                  180  84.375000
               2018                   876                  192  81.818182
               2019                   816                  156  84.705882
               2020                  1068                   72  92.682927